In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

In [2]:
url = 'https://www.ad.co.il/car?sp261=13922&rp264=2003,2022'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers)

In [3]:
if response.status_code == 200:
    print('ok')
    soup = BeautifulSoup(response.content, 'html.parser')
else:
    print(f'Request failed with status code: {response.status_code}')

ok


In [4]:
links = soup.find_all('a', href=True)
#links

In [19]:
list_for_df = []
for link in links:
        href = link['href']
        if href.startswith('/ad/16186') or href.startswith('/ad/15') or href.startswith('/ad/14') or href.startswith('/ad/13')  :
            car_url = f"https://www.ad.co.il{href}"
            car_page = requests.get(car_url)
            print(car_page)
             #create loop to stop the code if the status is not 200
            if car_page.status_code == 200: 
                prop = BeautifulSoup(car_page.content, "html.parser")            
                
                data_dict= {}
                prop_dict = {} 
                manufactor = "Lexsus"

                model =  prop.find("h2", {"class":"card-title"})
                model_str = model.get_text() if model else "None"
     
                Year = prop.find('p', {'class':'card-text'})
                Year_str = Year.get_text() if Year else "None"
                Year_int= int(Year_str)
        
                table = prop.find('table', {'class':'table table-sm mb-4'})
                for row in table.find_all('tr'):
                    columns= row.find_all('td')
                    key=columns[0].get_text(strip=True) if columns[0] else "None" 
                    value= columns[1].get_text(strip=True) if columns[1] else "None" 
                    data_dict[key]=value
                    
             
                hand_mum=data_dict['יד']
                hand_int= int(hand_mum)

                Gear=data_dict['ת. הילוכים']
                
                capacity_Engine=data_dict['נפח']
                capacity_Engine_1= capacity_Engine.split(",")
                capacity_Engine_2= capacity_Engine_1[0]+capacity_Engine_1[1]
                capacity_Engine_int= int(capacity_Engine_2)
                
                Engine_type=data_dict['סוג מנוע']
                
                Prev_ownership= data_dict.get('בעלות קודמת',"None")
                
                Curr_ownership=data_dict.get('בעלות נוכחית',"None")

                Area=data_dict.get('אזור', "None")

                City=data_dict.get('עיר',"None")

                Price= prop.find_all('h2', {'class':'card-title'})
                price_str= Price [1]
                price_str_1= price_str.get_text() if price_str else "None"
                if price_str_1 != "None":
                    price_int = re.findall(r'\d+(?:,\d+)*',price_str_1)
                    price_int = int(price_int[0].replace(',', ''))
                else:
                    price_int = "None"
                price_float= float(price_int)
                
                km=data_dict.get('ק"מ', "None")
                if km!= "None":
                    km_1= km.split(",")
                    km_2= km_1[0]+km_1[1]
                    km_int= int(km_2)
                else:
                    km_int= "None"
                

                color= data_dict.get('צבע', "None")
                
                
                images = prop.find_all('div', {'class':'card overflow-hidden'})
                number_of_images = len(images)
    
                start_date = prop.find_all('div', {'class':'px-3'})
                start_date_1 = start_date[0]
                start_date_1_str= start_date_1.get_text() if start_date_1 else "None"
                start_date_2 =start_date_1_str.split(':')
                start_date_3= start_date_2[1]
                date_object = datetime.strptime(start_date_3, ' %d/%m/%Y').date()

                end_date_1 = start_date[1]
                end_date_1_str= end_date_1.get_text() if end_date_1 else "None"
                end_date_2 =end_date_1_str.split(':')
                end_date_3= end_date_2[1]
                date_object_end = datetime.strptime(end_date_3, ' %d/%m/%Y').date()

                Description=prop.find_all('p', {'class':'text-word-break'})
                if len(Description) > 0:
                    Description_1 = Description[0]
                    Description_1= Description_1.get_text() 
                else:
                    Description_1 = 'None'
                    
                
                Test =data_dict.get('טסט עד', "None")
                if Test != "None":
                    date_object_Test = datetime.strptime(Test, '%m/%Y').date()
                    date_object_Test_end= date_object_Test + relativedelta(day=31)
                    today = date.today()
                    future = date_object_Test_end
                    diff_str = str(future - today)
                    diff_str_1= diff_str.split(' ')
                    diff_int_1= int(diff_str_1[0])
                else:
                    diff_int_1= "None"

              
                

                list_for_df.append({"manufactor": manufactor ,
                        "model": model_str,
                        "Year": Year_int,
                        "Hand": hand_int,
                        "Gear": Gear,
                        "capacity Engine": capacity_Engine_int,
                        "Engine_type": Engine_type,
                        "Prev_ownership": Prev_ownership,
                        "Curr_ownership": Curr_ownership,
                        "km": km_int,
                        "color": color,
                        "City": City,  
                        "Area": Area,
                        "images": number_of_images,
                        "Price": price_float,
                        "start_date": date_object,
                        "end_date" : date_object_end,
                        "Description" : Description_1,
                        "Test": diff_int_1 })
                
                #break
            else:
                break
       
df = pd.DataFrame(list_for_df)
df['Gear'] = df.Gear.astype('category')
df['Engine_type'] = df.Engine_type.astype('category')
df['Prev_ownership'] = df.Prev_ownership.astype('category')
df['Curr_ownership'] = df.Curr_ownership.astype('category')
df 

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


,manufactor,model,Year,Hand,Gear,capacity Engine,Engine_type,Prev_ownership,Curr_ownership,km,color,City,Area,images,Price,start_date,end_date,Description,Test
0,Lexsus,לקסוס RC,2008,4,אוטומטית,3200,גז,פרטית,פרטית,180000,לבן,בני ברק,רמת גן - גבעתיים,4,60000.0,2024-05-20,2024-06-10,None,232
1,Lexsus,לקסוס IS250,2008,8,אוטומטית,2500,בנזין,פרטית,פרטית,220000,לבן שנהב,קרית ים,קריות,3,37500.0,2023-07-20,2023-07-20,is250 2008 דגם פרימיום\nגמיש לרציניים :)\nצבע ...,None
2,Lexsus,לקסוס IS250,2008,4,אוטומטית,2500,בנזין,פרטית,פרטית,None,שחור,יציץ,מושבים בשפלה,3,44000.0,2023-07-13,2023-07-12,לקסוס מטופלת ברמה הכי גבוהה לפני חודשיים שמתי ...,None
3,Lexsus,לקסוס NX,2021,2,אוטומטית,2500,היברידי,None,None,26000,None,קרית ים,קריות,3,260000.0,2022-10-15,2022-10-15,טסט לשנה שלימה אחרי טיפול גדול כל התוספות מהחב...,None
4,Lexsus,לקסוס GS300,2008,3,אוטומטית,3000,בנזין,פרטית,פרטית,240000,אפור מטאלי,זמר,None,5,40000.0,2022-09-22,2022-09-22,לקסוס gs300 למכירה \nמנוע 3.0 v6\n250 כ״ס\nרמת...,None
5,Lexsus,לקסוס IS300h,2014,3,אוטומטית,2500,היבריד,פרטית,פרטית,219000,שחור,ברקת,בקעת אונו,3,82000.0,2022-09-14,2022-09-14,רכב מסוג לקסוס is300h 2014 בצבע שחור מדהים!! ז...,None
6,Lexsus,לקסוס CT200H,2012,4,אוטומטית,1800,היברידי,None,None,199000,None,ירושלים,ירושלים והסביבה,3,53000.0,2022-08-20,2022-08-20,רכב אמין!!!!מוכרים בצער רב בעקבות התרחבות המשפ...,None
7,Lexsus,לקסוס IS300H,2014,3,אוטומטית,2500,היברידי,None,None,175000,None,כאבול,None,3,73000.0,2022-07-09,2022-07-09,None,None
8,Lexsus,לקסוס GS300,2016,2,אוטומטית,2500,היבריד,פרטית,פרטית,100000,לבן פנינה,רחובות,נס ציונה - רחובות,3,149900.0,2022-02-27,2022-06-05,כחדשה! מטופלת בלקסוס. אחריות על המערכת ההיברידית,None
9,Lexsus,לקסוס RC,2018,1,אוטומטית,2000,בנזין,פרטית,פרטית,39000,שחור,אבן יהודה,מושבים בשרון,5,194000.0,2022-04-06,2022-05-21,כל התוספות שמורה מאוד .,None


In [20]:
df.to_csv('output4.csv', encoding='utf-8-sig', index=False)